In [19]:
import google.generativeai as genai
import os
import smtplib
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

In [41]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
model=genai.GenerativeModel("gemini-2.0-flash")

In [46]:
def sales_agent_1(user_prompt):
    prompt = f"You are a sales agent working for ComplAI, " \
             f"a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. " \
             f"You write professional, serious cold emails for the given user prompt: \"{user_prompt}\"."
    return model.generate_content(prompt).text


def sales_agent_2(user_prompt):
    prompt = f"You are a humorous, engaging sales agent working for ComplAI, " \
             f"a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. " \
             f"You write witty, engaging cold emails that are likely to get a response for the given user prompt: \"{user_prompt}\"."
    return model.generate_content(prompt).text


def sales_agent_3(user_prompt):
    prompt = f"You are a busy sales agent working for ComplAI, " \
             f"a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. " \
             f"You write concise, to the point cold emails for the given user prompt: \"{user_prompt}\"."
    return model.generate_content(prompt).text


In [47]:
sales_tools = {
    "Tool 1": sales_agent_1,
    "Tool 2": sales_agent_2,
    "Tool 3": sales_agent_3,
}


In [89]:
def subject_writer(email_text):
    prompt = f"You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response for the given email text.Do NOT include multiple options, explanations, or reasoning. Output ONLY the subject line for the email without the heading 'Subject:' : \"{email_text}\"."
    return model.generate_content(prompt).text

def html_formatter(email_text):
    prompt = f"You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design without any further explanations about it or why you have chose this color/font/layout etc. for the given email text: \"{email_text}\". And also do not print/include '''html''' in the output."
    return model.generate_content(prompt).text

In [90]:
def send_email_via_sendgrid(subject, html_body):
    sender_email = "kopika0208@gmail.com"
    message = Mail(
        from_email=sender_email,
        to_emails='kopika0208@gmail.com',
        subject=subject,
        html_content=html_body
    )

    try:
        sg = SendGridAPIClient(os.environ['SENDGRID_API_KEY'])
        response = sg.send(message)
        print("Email sent:", response.status_code)
    except Exception as e:
        print("SendGrid Error:", e)

In [91]:
email_tools = {
    "Tool 1": subject_writer,
    "Tool 2": html_formatter,
}

In [92]:
def email_manager_agent(email_text):
    tools = {
        "subject_writer": subject_writer,
        "html_formatter": html_formatter,
        "send_email_via_sendgrid": send_email_via_sendgrid
    }

    # Prompt the model to generate just the final email text — no reasoning or explanations
    prompt = f"""
You are an email_manager_agent at ComplAI.

Your job is to produce a professional cold sales email based on the input below.
Do NOT provide explanations, reasoning, or any text besides the email content.

Input email body to process:

--- BEGIN EMAIL BODY ---
{email_text}
--- END EMAIL BODY ---

Output only the email body text.
"""

    email_body = model.generate_content(prompt).text.strip()

    # Now call the tools to generate subject, html, and send the email
    subject = tools["subject_writer"](email_body)
    html_body = tools["html_formatter"](email_body)
    status = tools["send_email_via_sendgrid"](subject, html_body)

    return f"""✅ Email sent!

📬 Status: {status}
"""

In [93]:
def manager_agent(user_prompt):
    tool_outputs = {}

    # Pass user_prompt to each sales tool
    for name, tool in sales_tools.items():
        tool_outputs[name] = tool(user_prompt)

    # Main reasoning prompt with tool outputs and user context
    prompt = f"""
You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. 
You never generate emails yourself; you always use the tools. 
You try all 3 sales agent tools at least once before choosing the best one. 
You can use the tools multiple times if you're not satisfied with the results from the first try. 
You select the single best email using your own judgment of which email will be most effective. 
After picking the email, you hand off to the email_manager_agent to format and send the email.

Here is the user's request/context:
\"\"\"{user_prompt}\"\"\"

You have access to 3 sales agents as tools.

Tool 1 Output:
{tool_outputs['Tool 1']}

---

Tool 2 Output:
{tool_outputs['Tool 2']}

---

Tool 3 Output:
{tool_outputs['Tool 3']}

---

Now choose the best email from the above and pass it to the email_manager_agent to format and send.
Only pass the final chosen email content to the email_manager_agent.
"""
    final_email_text = model.generate_content(prompt).text
    return email_manager_agent(final_email_text)

In [94]:
user_prompt="Send out a cold sales email addressed to Dear CEO from Alice"

In [95]:
manager_agent(user_prompt)

Email sent: 202


'✅ Email sent!\n\n📬 Status: None\n'